In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from collections import OrderedDict
from operator import itemgetter
import re

In [2]:
ls storage/yelp_data/

Dataset_Challenge_Dataset_Agreement.pdf  tex_stars.csv
Yelp_Dataset_Challenge_Round_12.pdf      text8
health_business.csv                      y_model.out
health_business00.csv                    yelp_academic_dataset_business.json
health_business01.csv                    yelp_academic_dataset_checkin.json
health_business_NEW.csv                  yelp_academic_dataset_photo.json
health_clean0913.csv                     yelp_academic_dataset_review.csv
health_clean0918.csv                     yelp_academic_dataset_review.json
health_clean2.csv                        yelp_academic_dataset_review_full.csv
health_raw.csv                           yelp_academic_dataset_review_trim.csv
health_raw0913.csv                       yelp_academic_dataset_tip.json
health_raw0918.csv                       yelp_academic_dataset_user.json
json_to_csv_converter.py                 yelp_bus.csv
review_trim.csv                          yelp_dataset.tar.gz


In [3]:
path = 'storage/yelp_data/yelp_academic_dataset_business.json'

In [4]:
%%time
df = pd.read_json(path, lines=True)

CPU times: user 5.01 s, sys: 724 ms, total: 5.74 s
Wall time: 5.73 s


In [6]:
df.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV


In [7]:
%%time
data = df.copy()

CPU times: user 48 ms, sys: 8 ms, total: 56 ms
Wall time: 53.6 ms


In [8]:
data = data[['business_id','name','categories','state']]
data.head()

,business_id,name,categories,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,Minhas Micro Brewery,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",AB
1,AjEbIBw6ZFfln7ePHha9PA,CK'S BBQ & Catering,"Chicken Wings, Burgers, Caterers, Street Vendo...",NV
2,O8S5hYJ1SMc8fA4QBtVujA,La Bastringue,"Breakfast & Brunch, Restaurants, French, Sandw...",QC
3,bFzdJJ3wp3PZssNEsyU23g,Geico Insurance,"Insurance, Financial Services",AZ
4,8USyCYqpScwiNEb58Bt6CA,Action Engine,"Home & Garden, Nurseries & Gardening, Shopping...",AB


In [ ]:
#df2 = pd.DataFrame(data['categories'].str.split().values.tolist())

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 15 columns):
address         188593 non-null object
attributes      162807 non-null object
business_id     188593 non-null object
categories      188052 non-null object
city            188593 non-null object
hours           143791 non-null object
is_open         188593 non-null int64
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 21.6+ MB


In [11]:
df.head(2)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV


In [12]:
#list(df.columns)
df = df[[
 'business_id',
 'categories',
 'name','state']]

In [13]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Action Engine,AB


## Drop null categories

In [14]:
df[df.categories.isnull()]

,business_id,categories,name,state
288,EBzr465prEffkpmE8Mk5AA,None,Polish'd Nail Bar,ON
603,CN3BLZwfG4eqZjvKrIZoAg,None,Fuzziwigs Candy Factory,AZ
1091,Y76jGd_f889OT4oz9pLgZA,None,Center For Effective Living,OH
1179,yo8TQab_kYgAZm_DJkRjfA,None,A L Financial,AZ
1404,s_0zH6HExuWJeJjiZAtckA,None,CoCo Fresh Tea & Juice,ON
1887,v7BcP2B6IGR59ZrY4QUaYA,None,Ray's,NC
2683,FXY6bj0GTWZoiA_uRng6xQ,None,Vista Grove Prep Academy,AZ
3802,-J5SYRlH2hpv4uDs07lkqA,None,Aerus Electrolux,AZ
4455,nR6wsc5ld4bx6NTa0ogDwQ,None,Car Town U S A,NV
4752,IiJP9wxsnJBzUGLJKavHag,None,Hta Luxury and 4X4,AZ


In [15]:
len(df[df.categories.isnull()])

541

In [16]:
null_cat = df.categories.isnull()

In [17]:
len(df[~null_cat])

188052

In [18]:
df = df[~null_cat]
data = data[~null_cat]

In [19]:
df = df.reset_index(drop=True)
data = data.reset_index(drop=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188052 entries, 0 to 188051
Data columns (total 4 columns):
business_id    188052 non-null object
categories     188052 non-null object
name           188052 non-null object
state          188052 non-null object
dtypes: object(4)
memory usage: 5.7+ MB


In [21]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Action Engine,AB


## lowercase categories

In [22]:
df.categories = df.categories.str.lower()
data.categories = data.categories.str.lower()

In [23]:
df.categories = df.categories.str.split(',')
#data.categories = data.categories.str.split(',')

In [24]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"[tours, breweries, pizza, restaurants, foo...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"[chicken wings, burgers, caterers, street v...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"[breakfast & brunch, restaurants, french, s...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"[insurance, financial services]",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"[home & garden, nurseries & gardening, shopp...",Action Engine,AB


# create df from categories

In [25]:
# df2 = pd.DataFrame(data['categories'].str.split(',').values.tolist())
# df2.head()

In [26]:
counts = dict()

In [27]:
counts

{}

In [ ]:
# %%time
# for i in range(10):
#     for j in df.categories[i]:
#         j= j.strip()
#         print(j)

In [28]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"[tours, breweries, pizza, restaurants, foo...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"[chicken wings, burgers, caterers, street v...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"[breakfast & brunch, restaurants, french, s...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"[insurance, financial services]",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"[home & garden, nurseries & gardening, shopp...",Action Engine,AB


In [29]:
%%time
for i in range(len(df.categories)):
    for j in df.categories[i]:
        j= j.lstrip()
        counts[j] = counts.get(j, 0) + 1

CPU times: user 5.04 s, sys: 0 ns, total: 5.04 s
Wall time: 5.04 s


## total categories

In [30]:
len(counts)

1305

In [31]:
counts

{'tours': 639,
 'breweries': 508,
 'pizza': 6603,
 'restaurants': 57173,
 'food': 27118,
 'hotels & travel': 5808,
 'chicken wings': 2538,
 'burgers': 5126,
 'caterers': 2184,
 'street vendors': 304,
 'barbeque': 1716,
 'food trucks': 804,
 'event planning & services': 9774,
 'breakfast & brunch': 5023,
 'french': 977,
 'sandwiches': 6912,
 'cafes': 3056,
 'insurance': 869,
 'financial services': 2932,
 'home & garden': 6020,
 'nurseries & gardening': 615,
 'shopping': 30231,
 'local services': 12906,
 'automotive': 12656,
 'electronics repair': 611,
 'coffee & tea': 6936,
 'bakeries': 3509,
 'thai': 1393,
 'mexican': 4419,
 'flowers & gifts': 2058,
 'gift shops': 718,
 'japanese': 2566,
 'cajun/creole': 274,
 'southern': 493,
 'bars': 10853,
 'sports bars': 2055,
 'dive bars': 580,
 'nightlife': 12438,
 'pakistani': 397,
 'indian': 1417,
 'middle eastern': 1182,
 'beauty & spas': 18967,
 'barbers': 1932,
 'delis': 1835,
 'american (traditional)': 6659,
 'tapas/small plates': 428,
 'po

In [32]:
d = OrderedDict(sorted(counts.items(), key=itemgetter(1),reverse=True))

In [33]:
d

OrderedDict([('restaurants', 57173),
             ('shopping', 30231),
             ('food', 27118),
             ('beauty & spas', 18967),
             ('home services', 18634),
             ('health & medical', 16157),
             ('local services', 12906),
             ('automotive', 12656),
             ('nightlife', 12438),
             ('bars', 10853),
             ('event planning & services', 9774),
             ('active life', 9119),
             ('fashion', 7406),
             ('coffee & tea', 6936),
             ('sandwiches', 6912),
             ('hair salons', 6825),
             ('fast food', 6812),
             ('american (traditional)', 6659),
             ('pizza', 6603),
             ('home & garden', 6020),
             ('auto repair', 5877),
             ('hotels & travel', 5808),
             ('arts & entertainment', 5794),
             ('professional services', 5620),
             ('doctors', 5450),
             ('real estate', 5295),
             ('burgers', 512

In [34]:
cats = list(d.keys())

In [35]:
len(cats)

1305

In [36]:
cats

['restaurants',
 'shopping',
 'food',
 'beauty & spas',
 'home services',
 'health & medical',
 'local services',
 'automotive',
 'nightlife',
 'bars',
 'event planning & services',
 'active life',
 'fashion',
 'coffee & tea',
 'sandwiches',
 'hair salons',
 'fast food',
 'american (traditional)',
 'pizza',
 'home & garden',
 'auto repair',
 'hotels & travel',
 'arts & entertainment',
 'professional services',
 'doctors',
 'real estate',
 'burgers',
 'breakfast & brunch',
 'nail salons',
 'fitness & instruction',
 'italian',
 'mexican',
 'specialty food',
 'chinese',
 'american (new)',
 'pets',
 'hair removal',
 'bakeries',
 'dentists',
 'grocery',
 'skin care',
 'cafes',
 'education',
 'desserts',
 'financial services',
 'contractors',
 "women's clothing",
 'pet services',
 'general dentistry',
 'massage',
 'ice cream & frozen yogurt',
 'hotels',
 'japanese',
 'chicken wings',
 'day spas',
 'seafood',
 'salad',
 'apartments',
 'waxing',
 'gyms',
 'caterers',
 'tires',
 'sushi bars',
 

In [37]:
d['urgent care']

430

In [38]:
d['health & medical']

16157

In [39]:
#hh = df.categories.str.contains('health & medical', flags=re.IGNORECASE, regex=True)

In [40]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"[tours, breweries, pizza, restaurants, foo...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"[chicken wings, burgers, caterers, street v...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"[breakfast & brunch, restaurants, french, s...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"[insurance, financial services]",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"[home & garden, nurseries & gardening, shopp...",Action Engine,AB


In [41]:
#df2 = pd.DataFrame(df['categories'].str.split().values.tolist())

In [42]:
my_list = ['doctors','family practice','urgent care','obstetricians & gynecologists',
          'cosmetic surgeons',
           'internal medicine',
           'dermatologists',
           'surgeons',
           'ear nose & throat',
           'psychiatrists',
           'oncologist',
           'plastic surgeons',
           'gastroenterologist',
           'urologists',
           'emergency rooms',
           'pulmonologist',
           'spine surgeons',
           'pathologists',
           'neurotologists',
           'emergency medicine',
           'ultrasound imaging centers'
          ]

```
doctors: 5k
family practice: 711
oral surgeons: 681
urgent care: 430
obstetricians & gynecologists', 415
('cosmetic surgeons', 380),
('counseling & mental health', 367),
('internal medicine', 353),
('dermatologists', 275),
('rehabilitation center', 254),
('home health care', 226),
('laboratory testing', 168),
('surgeons', 144),
('ear nose & throat', 104),
('ear nose & throat', 104),
('psychiatrists', 86),
('oncologist', 50),
('plastic surgeons', 66),
('gastroenterologist', 62),
('urologists', 60),
('fertility', 57),
('emergency rooms', 48),
('health insurance offices', 30),
('pulmonologist', 27),
('spine surgeons', 24),
('pathologists', 4),
('neurotologists', 8),
('emergency medicine', 8),
('ultrasound imaging centers', 9),
```

In [43]:
my_list

['doctors',
 'family practice',
 'urgent care',
 'obstetricians & gynecologists',
 'cosmetic surgeons',
 'internal medicine',
 'dermatologists',
 'surgeons',
 'ear nose & throat',
 'psychiatrists',
 'oncologist',
 'plastic surgeons',
 'gastroenterologist',
 'urologists',
 'emergency rooms',
 'pulmonologist',
 'spine surgeons',
 'pathologists',
 'neurotologists',
 'emergency medicine',
 'ultrasound imaging centers']

In [ ]:
#df[df['categories'].isin(my_list)]

In [44]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"[tours, breweries, pizza, restaurants, foo...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"[chicken wings, burgers, caterers, street v...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"[breakfast & brunch, restaurants, french, s...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"[insurance, financial services]",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"[home & garden, nurseries & gardening, shopp...",Action Engine,AB


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188052 entries, 0 to 188051
Data columns (total 4 columns):
business_id    188052 non-null object
categories     188052 non-null object
name           188052 non-null object
state          188052 non-null object
dtypes: object(4)
memory usage: 5.7+ MB


In [ ]:
#df[df.isnull()]
#df2.head()

## search withing new categorie
`df2[df2[0].isin(my_list)]`: 2118 

In [ ]:
#data[data.categories.isin(['urologists'])]
#df[df2[0].isin(my_list)]
health1 = df[df2[0].isin(my_list)]

In [ ]:
health1.head()

In [46]:
data.categories

0         tours, breweries, pizza, restaurants, food, ho...
1         chicken wings, burgers, caterers, street vendo...
2         breakfast & brunch, restaurants, french, sandw...
3                             insurance, financial services
4         home & garden, nurseries & gardening, shopping...
5                                        coffee & tea, food
6                                            food, bakeries
7                                         restaurants, thai
8                                      mexican, restaurants
9                     flowers & gifts, gift shops, shopping
10                                    restaurants, japanese
11                      cajun/creole, southern, restaurants
12        bars, sports bars, dive bars, burgers, nightli...
13           restaurants, pakistani, indian, middle eastern
14                                   beauty & spas, barbers
15                           delis, restaurants, sandwiches
16        nightlife, bars, american (tra

In [47]:
%%time
health = data[data['categories'].apply(lambda x: pd.Series(x.split(', ')).isin(my_list).any())]

CPU times: user 56.4 s, sys: 0 ns, total: 56.4 s
Wall time: 56.4 s


In [48]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5656 entries, 78 to 187983
Data columns (total 4 columns):
business_id    5656 non-null object
name           5656 non-null object
categories     5656 non-null object
state          5656 non-null object
dtypes: object(4)
memory usage: 220.9+ KB


In [49]:
health.head()

,business_id,name,categories,state
78,y-4xTZNKVm8mAZpiXMS5ZA,"Lauren Byrne, MD","urologists, doctors, health & medical",AZ
139,STcSSjPOZ0FWtjvjqw99Dg,Black Mountain Orthopaedics,"doctors, orthopedists, health & medical",NV
165,WuiRF916fJl6aG5wC8qRAg,Advanced Brows,"doctors, eyebrow services, tattoo removal, bea...",NV
169,WnRtn80rdsWOa2nWZ6G1iA,Central Phoenix Women's Health Care,"obstetricians & gynecologists, health & medica...",AZ
183,SaR_j1ev_d55IBzORph_lQ,LASIK MD,"laser eye surgery/lasik, optometrists, ophthal...",QC


In [50]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5656 entries, 78 to 187983
Data columns (total 4 columns):
business_id    5656 non-null object
name           5656 non-null object
categories     5656 non-null object
state          5656 non-null object
dtypes: object(4)
memory usage: 220.9+ KB


## save business dataframe

In [51]:
health.to_csv('storage/yelp_data/health_business00.csv')

In [52]:
ls storage/yelp_data/

Dataset_Challenge_Dataset_Agreement.pdf  tex_stars.csv
Yelp_Dataset_Challenge_Round_12.pdf      text8
health_business.csv                      y_model.out
health_business00.csv                    yelp_academic_dataset_business.json
health_business01.csv                    yelp_academic_dataset_checkin.json
health_business_NEW.csv                  yelp_academic_dataset_photo.json
health_clean0913.csv                     yelp_academic_dataset_review.csv
health_clean0918.csv                     yelp_academic_dataset_review.json
health_clean2.csv                        yelp_academic_dataset_review_full.csv
health_raw.csv                           yelp_academic_dataset_review_trim.csv
health_raw0913.csv                       yelp_academic_dataset_tip.json
health_raw0918.csv                       yelp_academic_dataset_user.json
json_to_csv_converter.py                 yelp_bus.csv
review_trim.csv                          yelp_dataset.tar.gz
